In [1]:
from model import GPT
import numpy as np
import torch
from tqdm import tqdm
from copy import deepcopy

In [2]:
data = np.memmap('/data2/mengfanxu/nanoGPT/data/openwebtext/val.bin', dtype=np.uint16, mode='r')

In [3]:
batch_size = 16
block_size = 1024
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])

In [4]:
model = GPT.from_pretrained('/data2/mengfanxu/huggingface/gpt2-xl')

/home/mengfanxu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading weights from pretrained gpt: gpt2-xl
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 1555.97M


In [6]:
orthogonal_model = deepcopy(model)
state_dict = torch.load("orthogonal/gpt2-xl.pt")
orthogonal_model.load_state_dict(state_dict)

/tmp/ipykernel_2935651/3779957639.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("orthogonal/gpt2-xl.pt")


<All keys matched successfully>

In [7]:
logits, loss, qk_norm_list, v_norm_list = model(x)

In [8]:
orthogonal_logits, loss, orthogonal_qk_norm_list, orthogonal_v_norm_list = orthogonal_model(x)

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt

for layer in range(len(qk_norm_list)):
    plt.figure(figsize=(20,3))
    plt.plot(qk_norm_list[layer],label="vanila")
    plt.plot(orthogonal_qk_norm_list[layer], label='orthogonal')
    plt.ylim(0,50)
    plt.legend()
    plt.show()

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt

for layer in range(len(v_norm_list)):
    plt.figure(figsize=(20,3))
    plt.plot(v_norm_list[layer],label="vanila")
    plt.plot(orthogonal_v_norm_list[layer], label='orthogonal')
    plt.ylim(0,100)
    plt.legend()
    plt.show()

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt
for layer in range(len(v_norm_list)):
    plt.plot(v_norm_list[layer].reshape(-1,64).sort(dim=-1,descending=True)[0].reshape(-1),label="vanila")
    plt.plot(orthogonal_v_norm_list[layer].reshape(-1,64).sort(dim=-1,descending=True)[0].reshape(-1), label='orthogonal')
    #plt.ylim(0,50)
    plt.legend()
    plt.show()

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt
for layer in range(len(qk_norm_list)):
    plt.plot(qk_norm_list[layer].reshape(-1,64).sort(dim=-1,descending=True)[0].reshape(-1),label="vanila")
    plt.plot(orthogonal_qk_norm_list[layer].reshape(-1,64).sort(dim=-1,descending=True)[0].reshape(-1), label='orthogonal')
    #plt.ylim(0,500)
    plt.legend()
    plt.show()